# Generate Dataframe of Weather Information 
- Scrape historical weather infromation from https://www.wunderground.com
- Location: KBWI

In [59]:
import pickle
import numpy as np
import pandas as pd
import datetime
from datetime import datetime as dt
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
# Time
start_date = dt(2022, 2, 14)
end_date = dt(2023, 2, 12)
start_time = 330 # 05:30:00 
end_time = 1260 # 21:00:00
busi_date = pd.bdate_range(start=start_date, end=end_date).date  # 260 business dates

### 1. Web Scraping
Execute the following code blocks ONLY IF TSMO_df_weather_raw.pkl is not in place

In [56]:
# There are chances where the tables may not be fully loaded. 
# Therefore, it's necessary to return to dates where the data was not successfully scraped at first attempt.
# Therefore, the helper funciton is not directly used but modified as shown in the next code block to ensure weather info of all dates are captured.
def get_weather_df(date):
    date_str = date.strftime('%Y-%m-%d')
    url = f'https://www.wunderground.com/history/daily/KBWI/date/{date_str}'
    driver = webdriver.Safari()
    driver.get(url)
    tables = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
    result = pd.read_html(tables[1].get_attribute('outerHTML'))[0]
    driver.quit()
    result = result[~result.Time.isna()]
    result["Date"] = date
    # driver.quit()
    return result

In [63]:
dates = list(busi_date)
list_df_weather = []
while len(dates) > 0:
    # keep scraping until there's no date missed
    missed_dates = []
    for date in tqdm(dates):
        # list_df_weather.append(get_weather_df(date))

        date_str = date.strftime('%Y-%m-%d')
        url = f'https://www.wunderground.com/history/daily/KBWI/date/{date_str}'
        driver = webdriver.Safari()
        driver.get(url)
        try: 
            tables = WebDriverWait(driver,20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "table")))
            if len(tables) < 2:
                # tables may not be fully loaded
                missed_dates.append(date)
                driver.quit()
            else:
                result = pd.read_html(tables[1].get_attribute('outerHTML'))[0]
                driver.quit()
                result = result[~result.Time.isna()]
                result["Date"] = date

                list_df_weather.append(result)
        except:
            # may encounter overtime exception
            missed_dates.append(date)
            driver.quit()
    print(f"Remaining (missed): {len(missed_dates)} dates!")
    dates = missed_dates.copy()
    


100%|██████████| 260/260 [13:59<00:00,  3.23s/it]


Remaining (missed): 31 dates!


100%|██████████| 31/31 [01:28<00:00,  2.85s/it]


Remaining (missed): 1 dates!


100%|██████████| 1/1 [00:02<00:00,  2.25s/it]

Remaining (missed): 0 dates!


In [64]:
df_weather_raw = pd.concat(list_df_weather, axis=0)  # (7241, 11), covers all of 260 business dates

In [67]:
pickle.dump(df_weather_raw, open("./TSMO_df_weather_raw.pkl", "wb"))

### 2. Process Weather Dataframe
- Clean date and time to focus on business hours with 5-min granularity 
- Impute missing data

In [29]:
df_weather_raw = pickle.load(open("./TSMO_df_weather_raw.pkl", "rb"))

In [62]:
df_weather_raw.columns

Index(['Time', 'Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed',
       'Wind Gust', 'Pressure', 'Precip.', 'Condition', 'Date'],
      dtype='object')

In [63]:
df_weather_raw.index = df_weather_raw.apply(lambda x: dt.strptime(x["Date"].strftime(format="%Y-%m-%d ")+x["Time"], "%Y-%m-%d %I:%M %p"), axis=1)

In [64]:
df_weather_raw

,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Date
2022-02-14 01:54:00,1:54 AM,27 °F,17 °F,66 °%,W,3 °mph,0 °mph,29.91 °in,0.0 °in,Mostly Cloudy,2022-02-14
2022-02-14 02:54:00,2:54 AM,27 °F,17 °F,66 °%,WSW,6 °mph,0 °mph,29.92 °in,0.0 °in,Cloudy,2022-02-14
2022-02-14 03:54:00,3:54 AM,27 °F,16 °F,63 °%,NE,3 °mph,0 °mph,29.92 °in,0.0 °in,Light Snow,2022-02-14
2022-02-14 04:25:00,4:25 AM,26 °F,15 °F,63 °%,NE,5 °mph,0 °mph,29.92 °in,0.0 °in,Partly Cloudy,2022-02-14
2022-02-14 04:54:00,4:54 AM,25 °F,15 °F,66 °%,NE,6 °mph,0 °mph,29.92 °in,0.0 °in,Fair,2022-02-14
...,...,...,...,...,...,...,...,...,...,...,...
2023-01-30 20:54:00,8:54 PM,51 °F,39 °F,63 °%,SSW,3 °mph,0 °mph,29.94 °in,0.0 °in,Fair,2023-01-30
2023-01-30 21:54:00,9:54 PM,45 °F,39 °F,80 °%,CALM,0 °mph,0 °mph,29.93 °in,0.0 °in,Fair,2023-01-30
2023-01-30 22:54:00,10:54 PM,47 °F,39 °F,74 °%,CALM,0 °mph,0 °mph,29.93 °in,0.0 °in,Partly Cloudy,2023-01-30
2023-01-30 23:54:00,11:54 PM,46 °F,39 °F,76 °%,CALM,0 °mph,0 °mph,29.93 °in,0.0 °in,Partly Cloudy,2023-01-30


In [65]:
'''
temperature, humidity, wind speed, pressure, visibility, and hourly pre-cipitation, and a binary variable—pavement condition
'''
df_weather = pd.DataFrame()
df_weather["temperature"] = df_weather_raw.apply(lambda x: float(x["Temperature"].split()[0]), axis=1)
df_weather["dew_point"] = df_weather_raw.apply(lambda x: float(x["Dew Point"].split()[0]), axis=1)
df_weather["humidity"] = df_weather_raw.apply(lambda x: float(x["Humidity"].split()[0]), axis=1)
df_weather["wind_spd"] = df_weather_raw.apply(lambda x: float(x["Wind Speed"].split()[0]), axis=1)
df_weather["wind_gust"] = df_weather_raw.apply(lambda x: float(x["Wind Gust"].split()[0]), axis=1)
df_weather["pressure"] = df_weather_raw.apply(lambda x: float(x["Pressure"].split()[0]), axis=1)
df_weather["precipitation"] = df_weather_raw.apply(lambda x: float(x["Precip."].split()[0]), axis=1)
df_weather["condition"] = df_weather_raw["Condition"]

In [66]:
# Resampling and filtering busi-date and time
df_weather = df_weather.resample("1min").ffill()  # (521221, 7)
df_weather = df_weather[
            (pd.to_datetime(df_weather.index.date).isin(busi_date)) &   # be careful about the syntax here, dealing with index filtering is really annoying
            (df_weather.index.hour*60 + df_weather.index.minute >= start_time) &
            (df_weather.index.hour*60 + df_weather.index.minute < end_time) 
        ]  # (241800, 8)

In [67]:
df_weather.condition.unique()

array(['Fair', 'Mostly Cloudy', 'Partly Cloudy', 'Light Snow',
       'Heavy Snow', 'Cloudy', 'Heavy Rain / Windy', 'Light Rain / Windy',
       'Light Rain', 'Mostly Cloudy / Windy', 'Partly Cloudy / Windy',
       'Wintry Mix', 'Fog', 'Cloudy / Windy', 'Rain', 'Heavy Rain',
       'Shallow Fog', 'Mist', 'Light Drizzle', 'Light Snow / Windy',
       'Drizzle', 'Rain / Windy', 'Haze', 'Light Rain with Thunder',
       'Heavy T-Storm', 'Thunder', 'T-Storm', 'T-Storm / Windy',
       'Thunder / Windy', 'Heavy T-Storm / Windy', 'Patches of Fog',
       'Fair / Windy', 'Wintry Mix / Windy'], dtype=object)

In [68]:
df_weather.shape

(241800, 8)

In [69]:
pickle.dump(df_weather, open("./TSMO_df_weather.pkl", "wb"))

### 3. Finalize Dataframe of Weather and Time 
- Incorporate time embedding

In [70]:
# Code From Weiran
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
dr = pd.date_range(start='2019-01-01', end='2019-12-31')
cal = calendar()
holidays = cal.holidays(start=dr.min(), end=dr.max())
def dayofweek_encoder(row):
    if row['date'] in holidays or row['dow'] in [5,6]:
        row['dow_wkd_holiday'] = 1
    elif row['dow'] == 0:
        row['dow_mon'] = 1
    elif row['dow'] in [1,2,3]:
        row['dow_tue_thr'] = 1
    else:
        row['dow_fri'] = 1
    return row

def add_time_features(weather_df):
    weather_df['dow'] = weather_df.index.dayofweek
    weather_df['date'] = weather_df.index.date
    weather_df['ts'] = weather_df.index.hour*12+weather_df.index.minute//5 
    weather_df['month'] = weather_df.index.month - 1
    weather_df['week'] = weather_df.index.weekofyear - 1
    weather_df['mon_sin'] = np.sin(np.float64(2 * np.pi * weather_df.month/12))
    weather_df['mon_cos'] = np.cos(np.float64(2 * np.pi * weather_df.month/12))
    weather_df['week_sin'] = np.sin(np.float64(2 * np.pi * weather_df.week/52))
    weather_df['week_cos'] = np.cos(np.float64(2 * np.pi * weather_df.week/52))
    weather_df['ts_sin'] = np.sin(np.float64(2 * np.pi * weather_df.ts/288))
    weather_df['ts_cos'] = np.cos(np.float64(2 * np.pi * weather_df.ts/288))
    weather_df['dow_wkd_holiday'] = 0
    weather_df['dow_mon'] = 0
    weather_df['dow_tue_thr'] = 0
    weather_df['dow_fri'] = 0
    weather_df = weather_df.apply(dayofweek_encoder, axis=1)
    return weather_df.drop(['date','month','week','dow','ts'],axis=1)

In [71]:
df_weather_time = add_time_features(df_weather) # (241800, 18)

/var/folders/y2/1kk0np3x01qchz21plxx_m0w0000gn/T/ipykernel_9233/2717468186.py:22: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  weather_df['week'] = weather_df.index.weekofyear - 1


In [72]:
df_weather_time

,temperature,dew_point,humidity,wind_spd,wind_gust,pressure,precipitation,condition,mon_sin,mon_cos,week_sin,week_cos,ts_sin,ts_cos,dow_wkd_holiday,dow_mon,dow_tue_thr,dow_fri
2022-02-14 05:30:00,25.0,15.0,66.0,6.0,0.0,29.92,0.0,Fair,0.5,0.866025,0.663123,0.748511,0.991445,0.130526,0,1,0,0
2022-02-14 05:31:00,25.0,15.0,66.0,6.0,0.0,29.92,0.0,Fair,0.5,0.866025,0.663123,0.748511,0.991445,0.130526,0,1,0,0
2022-02-14 05:32:00,25.0,15.0,66.0,6.0,0.0,29.92,0.0,Fair,0.5,0.866025,0.663123,0.748511,0.991445,0.130526,0,1,0,0
2022-02-14 05:33:00,25.0,15.0,66.0,6.0,0.0,29.92,0.0,Fair,0.5,0.866025,0.663123,0.748511,0.991445,0.130526,0,1,0,0
2022-02-14 05:34:00,25.0,15.0,66.0,6.0,0.0,29.92,0.0,Fair,0.5,0.866025,0.663123,0.748511,0.991445,0.130526,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-10 20:55:00,49.0,30.0,48.0,16.0,0.0,29.99,0.0,Fair,0.5,0.866025,0.568065,0.822984,-0.722364,0.691513,0,0,0,1
2023-02-10 20:56:00,49.0,30.0,48.0,16.0,0.0,29.99,0.0,Fair,0.5,0.866025,0.568065,0.822984,-0.722364,0.691513,0,0,0,1
2023-02-10 20:57:00,49.0,30.0,48.0,16.0,0.0,29.99,0.0,Fair,0.5,0.866025,0.568065,0.822984,-0.722364,0.691513,0,0,0,1
2023-02-10 20:58:00,49.0,30.0,48.0,16.0,0.0,29.99,0.0,Fair,0.5,0.866025,0.568065,0.822984,-0.722364,0.691513,0,0,0,1


In [75]:
# It is noted that weather and date data in Cranberry contains the following 21 columns. 
# The weather information scraped from WeatherUnderground by Weiran is different from what I scrape from WeatherUnderground 
'''
'dewPt', 'gust', 'precip_hrly', 'precip_total', 'pressure', 'rh', 'snow_hrly', 'temp', 'vis', 'wspd', 'pressure_tend', 
'mon_sin', 'mon_cos', 'week_sin', 'week_cos', 'ts_sin', 'ts_cos'
'dow_fri', 'dow_mon', 'dow_tue_thr', 'dow_wkd_holiday',
'''

# In comparison, weather data in TSMO from WeatherUnderground is the following 18 columns
'''
'temperature', 'dew_point', 'humidity', 'wind_spd', 'wind_gust', 'pressure', 'precipitation', 'condition', 
'mon_sin', 'mon_cos', 'week_sin', 'week_cos', 'ts_sin', 'ts_cos', 
'dow_wkd_holiday', 'dow_mon', 'dow_tue_thr', 'dow_fri'
'''

"\n'temperature', 'dew_point', 'humidity', 'wind_spd', 'wind_gust', 'pressure', 'precipitation', 'condition', \n'mon_sin', 'mon_cos', 'week_sin', 'week_cos', 'ts_sin', 'ts_cos', \n'dow_wkd_holiday', 'dow_mon', 'dow_tue_thr', 'dow_fri'\n"

In [73]:
pickle.dump(df_weather_time, open("./TSMO_df_weather_time.pkl", "wb"))